In [1]:
import pandas as pd 
import numpy as np
import sqlalchemy
import pyodbc

import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.window as w
import pyspark.sql.types as t

In [2]:
spark = SparkSession.builder.appName('TheVoice-ETL').getOrCreate()

In [4]:
# Load into dataframes
call_type_mrr =         spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\call_type.parquet')
countries_mrr =         spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\countries.parquet')
customer_mrr =          spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\customer.parquet')
customer_invoice_mrr =  spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\customer_invoice.parquet')
customer_lines_mrr =    spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\customer_lines.parquet')
opfileopp_mrr =         spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\pfileopp.parquet')
package_catalog_mrr =   spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\package_catalog.parquet')
usage_main_mrr =        spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\usage_main.parquet')
xxCountryType_mrr =     spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\xxCountryType.parquet')

root
 |-- call_type_code: string (nullable = true)
 |-- call_type_desc: string (nullable = true)
 |-- priceperminuter: double (nullable = true)
 |-- call_type: string (nullable = true)

